In [78]:
import h5py
import pickle
import folium
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
with h5py.File('data/PEMS-BAY/pems-bay.h5', 'r') as file:

    axis0 = file['speed']['axis0'][:]               # Идентификаторы датчиков
    block0_items = file['speed']['block0_items'][:] # Идентификаторы датчиков
    axis1 = file['speed']['axis1'][:]               # Метки времени
    timestamps = pd.to_datetime(axis1)              # Преобразование меток времени в формат datetime
    speed_data = file['speed']['block0_values'][:]  # Данные замеров скорости

pems_bay = pd.DataFrame(speed_data, index=timestamps, columns=axis0)

# Открытие .pkl файла
with open('data/PEMS-BAY/adj_mx_bay.pkl', 'rb') as file:
    data = pickle.load(file, encoding='bytes')
    
node_ids = [x.decode('utf-8') for x in data[0]]                     # Получаем список id узлов из data[0]
adj_matrix = data[2]                                                # Получаем матрицу смежности из data[2]
adj_df = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids) # Создание DataFrame с использованием id узлов как индексов и названий колонок

distances_df = pd.read_csv('data/PEMS-BAY/distances_bay_2017.csv', header=None)
locations_df = pd.read_csv('data/PEMS-BAY/graph_sensor_locations_bay.csv', header=None)
distances_df.columns = ['from', 'to', 'distance']

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math

# Remove self-loops from adjacency matrix
adj_df.values[np.arange(325), np.arange(325)] = 0

# Normalize adjacency matrix
adj = adj_df.values
adj = adj + np.eye(325)
degrees = np.sum(adj, axis=1)
degrees_inv_sqrt = np.diag(1.0 / np.sqrt(degrees))
adj_norm = degrees_inv_sqrt @ adj @ degrees_inv_sqrt
adj_tensor = torch.FloatTensor(adj_norm)

# Normalize traffic data
scaler = StandardScaler()
pems_bay_scaled = scaler.fit_transform(pems_bay.values)

# Convert to tensor
X = torch.FloatTensor(pems_bay_scaled).permute(1, 0)  # Shape: [N, T]

# Parameters
N = 325  # Number of sensors
T = X.shape[1]
seq_len = 12
pre_len = 3
num_features = 1
emb_dim = 10
time_emb_dim = 12
hid_dim = 32
num_layers = 2
batch_size = 32

# Function to create sequences
def create_sequences(data, seq_len, pre_len):
    xs = []
    ys = []
    for i in range(len(data[0]) - seq_len - pre_len + 1):
        x = data[:, i:i+seq_len]
        y = data[:, i+seq_len:i+seq_len+pre_len]
        xs.append(x)
        ys.append(y)
    return torch.stack(xs), torch.stack(ys)

# Create sequences
X_seq, y_seq = create_sequences(X, seq_len, pre_len)

# Split into train, validation, and test sets
train_size = int(X_seq.size(0) * 0.7)
val_size = int(X_seq.size(0) * 0.1)
test_size = X_seq.size(0) - train_size - val_size

X_train, y_train = X_seq[:train_size], y_seq[:train_size]
X_val, y_val = X_seq[train_size:train_size+val_size], y_seq[train_size:train_size+val_size]
X_test, y_test = X_seq[train_size+val_size:], y_seq[train_size+val_size:]

# Custom Dataset class
class TrafficDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len
        self.N, self.T = X.shape
        self.time_steps = torch.arange(self.seq_len)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        t = self.time_steps
        return x, y, t

# Create DataLoaders
train_dataset = TrafficDataset(X_train, y_train, seq_len)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TrafficDataset(X_val, y_val, seq_len)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TrafficDataset(X_test, y_test, seq_len)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Fourier Transform
class FourierTransform(nn.Module):
    def __init__(self):
        super(FourierTransform, self).__init__()
    
    def forward(self, x):
        fft = torch.fft.fft(x, dim=1)
        magnitude = torch.abs(fft)
        phase = torch.angle(fft)
        return magnitude, phase

# Embedding Layers
class EmbeddingLayer(nn.Module):
    def __init__(self, N, emb_dim, seq_len, time_emb_dim):
        super(EmbeddingLayer, self).__init__()
        self.identity_emb = nn.Embedding(N, emb_dim)
        self.time_emb = nn.Embedding(seq_len, time_emb_dim)  # Time embedding for each time step
    
    def forward(self, x, t):
        identity = self.identity_emb(torch.arange(N, device=x.device))
        time = self.time_emb(t)
        return identity, time

# Dynamic Graph Construction
class DynamicGraph(nn.Module):
    def __init__(self, N, emb_dim, time_emb_dim):
        super(DynamicGraph, self).__init__()
        self.fc1 = nn.Linear(emb_dim + time_emb_dim, hid_dim)
        self.fc2 = nn.Linear(hid_dim, N)
    
    def forward(self, x, identity, time):
        x = torch.cat([x, identity, time], dim=1)
        x = F.relu(self.fc1(x))
        adj = self.fc2(x)
        adj = F.softmax(adj, dim=1)
        return adj

# Graph Convolution Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, adj):
        super(GraphConvolution, self).__init__()
        self.adj = adj
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()
    
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)
    
    def forward(self, input):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(self.adj, support) + self.bias
        return output

# Temporal Convolution Layer
class TemporalConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(TemporalConvolution, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=kernel_size//2)
    
    def forward(self, x):
        return self.conv(x)

# DFDGCN Model
class DFDGCN(nn.Module):
    def __init__(self, N, seq_len, num_features, adj, emb_dim, time_emb_dim, hid_dim, num_layers):
        super(DFDGCN, self).__init__()
        self.N = N
        self.seq_len = seq_len
        self.num_features = num_features
        self.emb_dim = emb_dim
        self.time_emb_dim = time_emb_dim
        self.hid_dim = hid_dim
        self.num_layers = num_layers
        
        self.fft = FourierTransform()
        self.embedding = EmbeddingLayer(N, emb_dim, seq_len, time_emb_dim)
        self.dynamic_graph = DynamicGraph(N, emb_dim, time_emb_dim)
        self.gconv_layers = nn.ModuleList([GraphConvolution(hid_dim, hid_dim, adj) for _ in range(num_layers)])
        self.tconv_layers = nn.ModuleList([TemporalConvolution(hid_dim, hid_dim, 3) for _ in range(num_layers)])
        self.fc = nn.Linear(hid_dim, num_features)
    
    def forward(self, x, t):
        # x: [N, seq_len, batch_size]
        # t: [seq_len]
        magnitude, phase = self.fft(x)
        identity, time = self.embedding(x, t)
        adj_dynamic = self.dynamic_graph(x, identity, time)
        
        h = torch.cat([magnitude, phase], dim=1)
        h = h.unsqueeze(2)  # [N, 2*seq_len, 1]
        
        for i in range(self.num_layers):
            h_g = self.gconv_layers[i](h[:, :, i])
            h_t = self.tconv_layers[i](h[:, :, i].unsqueeze(0))
            h = h_g + h_t
            h = F.relu(h)
        
        out = self.fc(h)
        return out

# Training function
def train(model, optimizer, criterion, dataloader, adj, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (x, y, t) in enumerate(dataloader):
            optimizer.zero_grad()
            # Forward pass
            output = model(x, t)
            # Compute loss
            loss = criterion(output, y)
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item()}')

# Evaluation function
def evaluate(model, criterion, dataloader, adj):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y, t in dataloader:
            output = model(x, t)
            loss = criterion(output, y)
            total_loss += loss.item()
    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Instantiate the model, optimizer, and criterion
model = DFDGCN(N, seq_len, num_features, adj_tensor, emb_dim, time_emb_dim, hid_dim, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train(model, optimizer, criterion, train_loader, adj_tensor, num_epochs=10)

# Evaluate the model on validation set
val_loss = evaluate(model, criterion, val_loader, adj_tensor)
print(f'Validation Loss: {val_loss}')

# Evaluate the model on test set
test_loss = evaluate(model, criterion, test_loader, adj_tensor)
print(f'Test Loss: {test_loss}')

ValueError: too many values to unpack (expected 2)

In [60]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class TrafficDataset(Dataset):
    def __init__(self, pems_bay, adj_df, seq_len, pre_len):
        """
        Инициализация датасета.
        
        :param pems_bay: pd.DataFrame, временные ряды трафика (в формате (время, узлы)).
        :param adj_df: pd.DataFrame, матрица смежности.
        :param seq_len: int, длина окна видимости (временного ряда для обучения).
        :param pre_len: int, длина окна предсказания.
        """
        self.pems_bay = pems_bay.values  # Преобразуем DataFrame в numpy array
        self.adj_df = torch.tensor(adj_df.values, dtype=torch.float32)  # Сразу преобразуем матрицу смежности в тензор
        self.seq_len = seq_len
        self.pre_len = pre_len
        self.num_nodes = self.pems_bay.shape[1]  # Количество узлов (датчиков)
        self.timestamps = pd.to_datetime(pems_bay.index)  # Преобразуем индексы в datetime

    def __len__(self):
        """
        Возвращает количество доступных временных окон.
        """
        return len(self.pems_bay) - self.seq_len - self.pre_len + 1

    def __getitem__(self, idx):
        """
        Возвращает один батч данных.
        
        :param idx: int, индекс окна.
        :return: dict, словарь с данными для модели.
        """
        # Временной ряд для обучения
        x = self.pems_bay[idx:idx + self.seq_len]
        # Целевые данные (предсказание)
        y = self.pems_bay[idx + self.seq_len:idx + self.seq_len + self.pre_len]
        
        # Временные метки для конца обучающего окна
        timestamp = self.timestamps[idx + self.seq_len - 1]
        day_of_week = timestamp.weekday()
        hour_of_day = timestamp.hour
        
        # Преобразуем данные в тензоры
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        
        return {
            'X_t': x,  # Входные данные (временные ряды)
            'y': y,  # Целевые данные (предсказание)
            'week_idx': torch.tensor(day_of_week, dtype=torch.long),  # День недели
            'day_idx': torch.tensor(hour_of_day, dtype=torch.long),  # Час дня
            'adj': self.adj_df  # Матрица смежности
        }


# Пример использования
if __name__ == "__main__":
    
    # Параметры
    seq_len = 12  # Длина окна видимости
    pre_len = 3  # Длина окна предсказания
    
    # Создание датасета
    dataset = TrafficDataset(pems_bay, adj_df, seq_len, pre_len)
    
    # Создание DataLoader
    batch_size = 32
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Пример вывода батча
    for batch in dataloader:
        print("X_t shape:", batch['X_t'].shape)     # [batch_size, seq_len, num_nodes]
        print("y shape:", batch['y'].shape)         # [batch_size, pre_len, num_nodes]
        print("week_idx:", batch['week_idx'])       # [batch_size]
        print("day_idx:", batch['day_idx'])         # [batch_size]
        print("adj shape:", batch['adj'].shape)     # [num_nodes, num_nodes]
        break

X_t shape: torch.Size([32, 12, 325])
y shape: torch.Size([32, 3, 325])
week_idx: tensor([2, 1, 1, 0, 4, 1, 0, 0, 1, 3, 6, 4, 1, 0, 4, 6, 5, 3, 2, 4, 6, 3, 0, 4,
        0, 0, 4, 3, 0, 4, 6, 3])
day_idx: tensor([ 5, 11, 20, 17, 15, 16, 23, 18, 23, 15, 19,  4,  5, 17, 18, 19,  0, 15,
        17,  8,  0, 20,  9,  8, 22, 11,  4,  7,  8, 20, 11, 21])
adj shape: torch.Size([32, 325, 325])


In [61]:
import torch
import torch.nn as nn
import torch.fft
import logging

# Настройка логгера
def setup_logger(log_file_path, terminal=False, log_level=logging.INFO, console_log_level=logging.INFO):
    """
    Настройка логгера для записи в файл и, при необходимости, в консоль.

    Аргументы:
        log_file_path (str): Путь к файлу для записи логов.
        terminal (bool): Включить ли вывод логов в консоль.
        log_level (int): Уровень логирования для файла (по умолчанию INFO).
        console_log_level (int): Уровень логирования для консоли (по умолчанию INFO).

    Возвращает:
        logger: Настроенный объект логгера.
    """
    logger = logging.getLogger("DFDGCN_Logger")
    
    # Удаляем все существующие обработчики, чтобы избежать дублирования
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    logger.setLevel(log_level)
    
    # Формат сообщений
    formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
    
    # Файловый хэндлер
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setFormatter(formatter)
    file_handler.setLevel(log_level)
    logger.addHandler(file_handler)

    # Консольный хэндлер (если terminal=True)
    if terminal:
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        console_handler.setLevel(console_log_level)
        logger.addHandler(console_handler)
    
    return logger

# Функция для логирования shape тензоров
def log_tensor_shape(logger, tensor, tensor_name):
    if tensor is not None:
        # Форматирование строки для выравнивания torch.Size([]) справа
        shape_str = f"{tensor.shape}".rjust(30)  # Выравнивание по правому краю
        logger.info(f"{tensor_name:<30} shape: {shape_str}")
    else:
        logger.info(f"{tensor_name} is None")

# Модель DFDGCN
class DFDGCN(nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_weeks, num_days, K, seq_len, pre_len, logger):
        super(DFDGCN, self).__init__()
        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.K = K
        self.seq_len = seq_len
        self.pre_len = pre_len  # Добавляем pre_len
        self.logger = logger

        # Identity Embedding
        self.identity_embedding = nn.Embedding(num_nodes, embedding_dim)
        
        # Time Embeddings
        self.week_embedding = nn.Embedding(num_weeks, embedding_dim)
        self.day_embedding = nn.Embedding(num_days, embedding_dim)
        
        # Linear layers for Fourier transformed data and time embeddings
        self.W_F = nn.Linear(1, embedding_dim)
        self.W_T = nn.Linear(2 * embedding_dim, embedding_dim)
        
        # 1x1 Convolution
        self.conv1x1 = nn.Conv1d(3 * embedding_dim, embedding_dim, kernel_size=1)
        
        # Adjacency matrix learning
        self.W_adj = nn.Parameter(torch.randn(embedding_dim, embedding_dim))
        
        # Graph convolution weights
        self.W_k1 = [nn.Linear(seq_len, embedding_dim) for _ in range(K + 1)]
        self.W_k2 = [nn.Linear(seq_len, embedding_dim) for _ in range(K + 1)]
        self.W_k3 = [nn.Linear(seq_len, embedding_dim) for _ in range(K + 1)]
        
        # Output layer to match target dimension
        self.output_layer = nn.Linear(embedding_dim, pre_len)  # Изменяем размерность выходного слоя

        # Инициализация A_adt как параметра модели
        self.A_adt = nn.Parameter(torch.rand(num_nodes, num_nodes))


    def apply_fft(self, x):
        fft_x = torch.fft.fft(x, dim=-1)
        return fft_x.abs()

    def compute_DE_t(self, x_t, week_idx, day_idx):
        fft_x_t = self.apply_fft(x_t)
        fft_x_t = fft_x_t.mean(dim=-1)
        log_tensor_shape(self.logger, fft_x_t, "FFT(X_t).mean")
        
        E_t = self.identity_embedding(torch.arange(self.num_nodes, device=x_t.device))
        log_tensor_shape(self.logger, E_t, "Identity Embedding")
        
        T_W = self.week_embedding(week_idx)
        T_D = self.day_embedding(day_idx)
        T = torch.cat((T_W, T_D), dim=-1)
        log_tensor_shape(self.logger, T, "Time Embedding (concat)")
        T = self.W_T(T)
        log_tensor_shape(self.logger, T, "Time Embedding (W_T)")
        
        F_t = fft_x_t.unsqueeze(-1)
        log_tensor_shape(self.logger, F_t, "FFT(X_t).unsqueeze")
        F_t = self.W_F(F_t).squeeze(-1)
        log_tensor_shape(self.logger, F_t, "Fourier Embedding (W_F)")
        
        # Перестановка F_t для совпадения размеров
        F_t = F_t.unsqueeze(1).expand(x_t.size(0), self.num_nodes, self.seq_len, self.embedding_dim)
        F_t = F_t.permute(0, 2, 1, 3).reshape(x_t.size(0), self.seq_len, self.num_nodes * self.embedding_dim)
        
        E_t_expanded = E_t.unsqueeze(0).expand(x_t.size(0), self.num_nodes, self.embedding_dim)
        log_tensor_shape(self.logger, E_t_expanded, "Identity Embedding (expanded)")
        T_expanded = T.unsqueeze(1).expand(x_t.size(0), self.num_nodes, self.embedding_dim)
        log_tensor_shape(self.logger, T_expanded, "Time Embedding (expanded)")
        
        # Конкатенация
        DE_t = torch.cat((F_t, E_t_expanded, T_expanded), dim=-1)
        log_tensor_shape(self.logger, DE_t, "DE_t (concat)")
        DE_t = torch.relu(DE_t)  # Добавляем активацию
        DE_t = torch.nn.functional.normalize(DE_t, p=2, dim=-1)  # Нормализация
        DE_t = DE_t.permute(0, 2, 1)
        log_tensor_shape(self.logger, DE_t, "DE_t (permute)")
        DE_t = self.conv1x1(DE_t)
        log_tensor_shape(self.logger, DE_t, "DE_t (conv1x1)")
        DE_t = DE_t.permute(0, 2, 1)
        log_tensor_shape(self.logger, DE_t, "DE_t (final)")
        return DE_t

    def compute_A_D(self, DE_t):
        A = torch.einsum('bne, ek->bnk', (DE_t, self.W_adj))
        A = torch.relu(A)
        A = torch.softmax(A, dim=-1)
        return A

    def graph_convolution(self, X_t, P, A_adt, A_D):
        Z_t = 0
        for k in range(self.K + 1):
            P_k = torch.matrix_power(P, k)
            log_tensor_shape(self.logger, P_k, f"P^{k}")
            A_adt_k = torch.matrix_power(A_adt, k)
            log_tensor_shape(self.logger, A_adt_k, f"A_adt^{k}")
            A_D_k = torch.matrix_power(A_D, k)
            log_tensor_shape(self.logger, A_D_k, f"A_D^{k}")
            
            term1 = (P_k @ X_t) @ self.W_k1[k].weight.t()
            log_tensor_shape(self.logger, term1, f"Term1 (P_k @ X_t)")
            term2 = (A_adt_k @ X_t) @ self.W_k2[k].weight.t()
            log_tensor_shape(self.logger, term2, f"Term2 (A_adt_k @ X_t)")
            term3 = (A_D_k @ X_t) @ self.W_k3[k].weight.t()
            log_tensor_shape(self.logger, term3, f"Term3 (A_D_k @ X_t)")
            Z_t += term1 + term2 + term3
        log_tensor_shape(self.logger, Z_t, "Z_t (graph convolution)")
        return Z_t

    def forward(self, X_t, week_idx, day_idx, P, A_adt):
        log_tensor_shape(self.logger, X_t, "X_t (input)")
        DE_t = self.compute_DE_t(X_t, week_idx, day_idx)
        A_D = self.compute_A_D(DE_t)
        Z_t = self.graph_convolution(X_t, P, A_adt, A_D)
        Z_t = self.output_layer(Z_t)  # [batch_size, num_nodes, pre_len]
        log_tensor_shape(self.logger, Z_t, "Z_t (output)")
        return Z_t

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DFDGCN(nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_weeks, num_days, K, seq_len, pre_len, logger):
        super(DFDGCN, self).__init__()
        self.num_nodes = num_nodes
        self.embedding_dim = embedding_dim
        self.K = K
        self.seq_len = seq_len
        self.pre_len = pre_len
        self.logger = logger

        # Identity Embedding (Эмбеддинг идентификаторов)
        self.identity_embedding = nn.Embedding(num_nodes, embedding_dim)
        
        # Time Embeddings (Эмбеддинги времени)
        self.week_embedding = nn.Embedding(num_weeks, embedding_dim)
        self.day_embedding = nn.Embedding(num_days, embedding_dim)
        
        # Линейные слои для преобразованных данных Фурье и эмбеддингов времени
        self.W_F = nn.Linear(seq_len, embedding_dim)
        self.W_T = nn.Linear(2 * embedding_dim, embedding_dim)
        
        # 1x1 Свертка
        self.conv1x1 = nn.Conv1d(3 * embedding_dim, embedding_dim, kernel_size=1)
        
        # Обучаемый параметр для матрицы смежности
        self.W_adj = nn.Parameter(torch.randn(embedding_dim, embedding_dim))
        
        # Веса для графовой свертки
        self.W_k1 = nn.ModuleList([nn.Linear(embedding_dim, embedding_dim) for _ in range(K + 1)])
        self.W_k2 = nn.ModuleList([nn.Linear(embedding_dim, embedding_dim) for _ in range(K + 1)])
        self.W_k3 = nn.ModuleList([nn.Linear(embedding_dim, embedding_dim) for _ in range(K + 1)])
        
        # Выходной слой для соответствия целевой размерности
        self.output_layer = nn.Linear(embedding_dim, pre_len)
        
        # Предопределенный граф P и самоадаптивный граф A_adt
        self.P = nn.Parameter(torch.randn(num_nodes, num_nodes))
        self.A_adt = nn.Parameter(torch.randn(num_nodes, num_nodes))

    def apply_fft(self, x):
        # Применение FFT и вычисление модуля
        fft_x = torch.fft.fft(x, dim=-1)
        fft_x = fft_x.abs()
        return fft_x

    def compute_DE_t(self, x_t, week_idx, day_idx):
        # Применение FFT и получение модуля
        fft_x_t = self.apply_fft(x_t)
        fft_x_t = fft_x_t.mean(dim=-1)
        
        # Identity Embedding (Эмбеддинг идентификаторов)
        E_t = self.identity_embedding(torch.arange(self.num_nodes, device=x_t.device))
        
        # Time Embedding (Эмбеддинг времени)
        T_W = self.week_embedding(week_idx)
        T_D = self.day_embedding(day_idx)
        T = torch.cat((T_W, T_D), dim=-1)
        T = self.W_T(T)
        
        # Обработка данных Фурье
        F_t = self.W_F(fft_x_t)
        
        # Объединение эмбеддингов
        DE_t = torch.cat((F_t, E_t, T), dim=-1)
        
        # 1x1 Свертка
        DE_t = DE_t.permute(0, 2, 1)
        DE_t = self.conv1x1(DE_t)
        DE_t = DE_t.permute(0, 2, 1)
        
        return DE_t

    def compute_A_D(self, DE_t):
        # Вычисление матрицы смежности
        A_logits = torch.matmul(DE_t, self.W_adj)
        A_logits = A_logits.permute(0, 2, 1)
        A_D = F.softmax(A_logits, dim=-1)
        return A_D

    def graph_convolution(self, X_t, P, A_adt, A_D):
        Z_t = 0
        for k in range(self.K + 1):
            P_k = torch.matrix_power(P, k)
            A_adt_k = torch.matrix_power(A_adt, k)
            A_D_k = torch.matrix_power(A_D, k)
            
            term1 = P_k @ X_t @ self.W_k1[k].weight.t()
            term2 = A_adt_k @ X_t @ self.W_k2[k].weight.t()
            term3 = A_D_k @ X_t @ self.W_k3[k].weight.t()
            Z_t += term1 + term2 + term3
        return Z_t

    def forward(self, X_t, week_idx, day_idx):
        DE_t = self.compute_DE_t(X_t, week_idx, day_idx)
        A_D = self.compute_A_D(DE_t)
        Z_t = self.graph_convolution(X_t, self.P, self.A_adt, A_D)
        Z_t = self.output_layer(Z_t)
        return Z_t

In [72]:
import torch
import numpy as np

def generate_synthetic_data(num_nodes, seq_len, pre_len, num_samples):
    """
    Генерация синтетических данных для модели DFDGCN.
    
    :param num_nodes: Количество узлов (датчиков) в графе.
    :param seq_len: Длина входной последовательности (количество временных шагов).
    :param pre_len: Длина предсказания (количество временных шагов для предсказания).
    :param num_samples: Количество сэмплов (батчей) для генерации.
    :return: Словарь с данными: X (входные данные), week_idx (индексы дней недели), day_idx (индексы часов дня), P (предопределенный граф).
    """
    # Генерация входных данных X
    X = np.random.rand(num_samples, num_nodes, seq_len)  # [batch_size, num_nodes, seq_len]
    
    # Генерация индексов дней недели и часов дня
    week_idx = np.random.randint(0, 7, size=(num_samples,))  # [batch_size]
    day_idx = np.random.randint(0, 24, size=(num_samples,))  # [batch_size]
    
    # Генерация предопределенного графа P
    P = np.random.rand(num_nodes, num_nodes)  # [num_nodes, num_nodes]
    P = P / np.sum(P, axis=1, keepdims=True)  # Нормализация строк
    
    # Преобразование данных в тензоры PyTorch
    X = torch.tensor(X, dtype=torch.float32)
    week_idx = torch.tensor(week_idx, dtype=torch.long)
    day_idx = torch.tensor(day_idx, dtype=torch.long)
    P = torch.tensor(P, dtype=torch.float32)
    
    return {
        'X': X,
        'week_idx': week_idx,
        'day_idx': day_idx,
        'P': P
    }

# Пример использования
num_nodes = 10  # Количество узлов (датчиков)
seq_len = 12    # Длина входной последовательности
pre_len = 6     # Длина предсказания
num_samples = 32  # Количество сэмплов (батчей)

data = generate_synthetic_data(num_nodes, seq_len, pre_len, num_samples)

# Вывод сгенерированных данных
print("Входные данные X:", data['X'].shape)
print("Индексы дней недели week_idx:", data['week_idx'].shape)
print("Индексы часов дня day_idx:", data['day_idx'].shape)
print("Предопределенный граф P:", data['P'].shape)

# Инициализация модели
model = DFDGCN(num_nodes=num_nodes, embedding_dim=10, num_weeks=7, num_days=24, K=2, seq_len=seq_len, pre_len=pre_len, logger=None)

# Передача данных в модель
output = model(data['X'], data['week_idx'], data['day_idx'])

# Вывод результата
print("Результат предсказания:", output.shape)

Входные данные X: torch.Size([32, 10, 12])
Индексы дней недели week_idx: torch.Size([32])
Индексы часов дня day_idx: torch.Size([32])
Предопределенный граф P: torch.Size([10, 10])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x32 and 12x10)

In [58]:
# Параметры
num_nodes = 10
seq_len = 12
pre_len = 3  # Окно предсказания
batch_size = 32
embedding_dim = 10
num_weeks = 7
num_days = 24
K = 2

# Генерация данных
X_t = torch.randn(batch_size, num_nodes, seq_len)
y = torch.randn(batch_size, num_nodes, pre_len)  # Целевые данные с размером pre_len
week_idx = torch.randint(0, num_weeks, (batch_size,))
day_idx = torch.randint(0, num_days, (batch_size,))
P = torch.rand(num_nodes, num_nodes)
A_adt = torch.rand(num_nodes, num_nodes)

In [68]:
# Настройка логгера с выводом в консоль
logger = setup_logger("dfdgnn_log.txt", terminal=False, log_level=logging.DEBUG, console_log_level=logging.DEBUG)

# Создание модели
model = DFDGCN(num_nodes, embedding_dim, num_weeks, num_days, K, seq_len, pre_len, logger)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Обучение
num_epochs = 1
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(X_t, week_idx, day_idx)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 32 but got size 10 for tensor number 1 in the list.

### model 2

In [18]:
import torch
from torch import nn
import torch.nn.functional as F
import logging


# Настройка логгера
def setup_logger(log_file_path, terminal=False, log_level=logging.INFO, console_log_level=logging.INFO):
    logger = logging.getLogger("DFDGCN_Logger")
    for handler in logger.handlers[:]:  # Удаляем существующие обработчики, если они есть
        logger.removeHandler(handler)
    logger.setLevel(log_level)

    formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setFormatter(formatter)
    file_handler.setLevel(log_level)
    logger.addHandler(file_handler)

    if terminal:
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        console_handler.setLevel(console_log_level)
        logger.addHandler(console_handler)

    return logger


# Логирование размеров тензоров
def log_tensor_shape(logger, tensor, tensor_name):
    if tensor is not None:
        logger.info(f"{tensor_name:<30} shape: {tensor.shape}")
    else:
        logger.info(f"{tensor_name:<30} is None")


# Основные компоненты модели
class FourierTransform(nn.Module):
    def __init__(self, seq_len, emb_dim, logger=None):
        super(FourierTransform, self).__init__()
        self.seq_len = seq_len
        self.emb_dim = emb_dim
        self.embedding = nn.Linear(seq_len // 2 + 1, emb_dim)
        self.logger = logger

    def forward(self, x):
        if self.logger:
            log_tensor_shape(self.logger, x, "Input to FourierTransform")
        x = torch.fft.rfft(x, dim=-1)
        x = torch.abs(x)
        x = F.normalize(x, p=2, dim=-1)
        x = self.embedding(x)
        if self.logger:
            log_tensor_shape(self.logger, x, "Output of FourierTransform")
        return x


class IdentityEmbedding(nn.Module):
    def __init__(self, num_nodes, emb_dim, logger=None):
        super(IdentityEmbedding, self).__init__()
        self.embedding = nn.Parameter(torch.randn(num_nodes, emb_dim))
        self.logger = logger

    def forward(self):
        if self.logger:
            log_tensor_shape(self.logger, self.embedding, "IdentityEmbedding")
        return self.embedding


class TimeEmbedding(nn.Module):
    def __init__(self, seq_len, time_emb_dim, logger=None):
        super(TimeEmbedding, self).__init__()
        self.day_embedding = nn.Embedding(7, time_emb_dim)
        self.hour_embedding = nn.Embedding(24, time_emb_dim)
        self.seq_len = seq_len
        self.logger = logger

    def forward(self, day_of_week, hour_of_day):
        day_emb = self.day_embedding(day_of_week)
        hour_emb = self.hour_embedding(hour_of_day)
        out = torch.cat([day_emb, hour_emb], dim=-1)
        if self.logger:
            log_tensor_shape(self.logger, out, "Output of TimeEmbedding")
        return out


class DynamicGraph(nn.Module):
    def __init__(self, emb_dim, hidden_dim, logger=None):
        super(DynamicGraph, self).__init__()
        input_dim = 4 * emb_dim  # Учитываем freq_emb, identity_emb, time_emb (два time_emb)
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)
        self.logger = logger

    def forward(self, freq_emb, identity_emb, time_emb):
        # Добавляем batch-измерение в identity_emb
        identity_emb = identity_emb.unsqueeze(0).repeat(freq_emb.size(0), 1, 1)
        if self.logger:
            log_tensor_shape(self.logger, identity_emb, "IdentityEmbedding with batch")

        # Объединение всех признаков
        combined = torch.cat([freq_emb, identity_emb, time_emb], dim=-1)  # [batch_size, num_nodes, 4 * emb_dim]
        if self.logger:
            log_tensor_shape(self.logger, combined, "Combined features for DynamicGraph")

        # Применяем полносвязный слой
        combined = self.activation(self.fc(combined))  # [batch_size, num_nodes, hidden_dim]

        # Генерация матрицы смежности
        adj_matrix = torch.bmm(combined, combined.transpose(1, 2))  # [batch_size, num_nodes, num_nodes]
        adj_matrix = self.softmax(F.relu(adj_matrix))
        if self.logger:
            log_tensor_shape(self.logger, adj_matrix, "Dynamic adjacency matrix")
        return adj_matrix



class GraphConv(nn.Module):
    def __init__(self, in_channels, out_channels, supports_len, logger=None):
        super(GraphConv, self).__init__()
        self.supports_len = supports_len
        self.conv_weights = nn.Parameter(torch.randn(supports_len, in_channels, out_channels))
        self.logger = logger

    def forward(self, x, supports):
        out = 0
        for i, A in enumerate(supports):
            partial_out = torch.einsum('bcn,cnm->bcm', x, A @ self.conv_weights[i])
            out += partial_out
            if self.logger:
                log_tensor_shape(self.logger, partial_out, f"GraphConv output for support {i}")
        return out


class DFDGCN(nn.Module):
    def __init__(self, num_nodes, in_dim, out_dim, seq_len, emb_dim, hidden_dim, supports=None, logger=None):
        super(DFDGCN, self).__init__()
        self.num_nodes = num_nodes
        self.seq_len = seq_len

        # Настройка логгера
        self.logger = logger

        # Подмодули
        self.fourier = FourierTransform(seq_len, emb_dim, logger)
        self.identity = IdentityEmbedding(num_nodes, emb_dim, logger)
        self.time = TimeEmbedding(seq_len, emb_dim, logger)
        self.dynamic_graph = DynamicGraph(emb_dim * 3, hidden_dim, logger)
        self.graph_conv = GraphConv(in_dim, out_dim, supports_len=1 + (1 if supports is not None else 0), logger=logger)

        # Инициализация графов
        self.supports = [supports] if supports is not None else []

    def forward(self, x, day_of_week, hour_of_day):
        if self.logger:
            log_tensor_shape(self.logger, x, "Input to DFDGCN")
        freq_emb = self.fourier(x)
        identity_emb = self.identity()
        time_emb = self.time(day_of_week, hour_of_day)

        dynamic_adj = self.dynamic_graph(freq_emb, identity_emb, time_emb)
        all_supports = self.supports + [dynamic_adj]

        x = x.unsqueeze(1)
        out = self.graph_conv(x, all_supports)
        if self.logger:
            log_tensor_shape(self.logger, out, "Output of DFDGCN")
        return out

In [ ]:
# Настройка логгера
logger = setup_logger("dfdgcn_logs.txt", terminal=True)

# Инициализация данных
batch_size = 32
num_nodes = 100
seq_len = 12
features = 1

time_series = torch.rand(batch_size, num_nodes, seq_len)
norm_adj = torch.rand(num_nodes, num_nodes)
day_of_week = torch.randint(0, 7, (batch_size, num_nodes))
hour_of_day = torch.randint(0, 24, (batch_size, num_nodes))

# Инициализация модели
model = DFDGCN(num_nodes=num_nodes, in_dim=features, out_dim=3, seq_len=seq_len,
               emb_dim=10, hidden_dim=30, supports=norm_adj, logger=logger)

# Прямой проход
output = model(time_series, day_of_week, hour_of_day)

## model 3

In [43]:
import h5py
import pickle
import folium
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
with h5py.File('data/raw_data_without_large_scale_datasets/raw_data/PEMS-BAY/PEMS-BAY.h5', 'r') as file:

    axis0 = file['speed']['axis0'][:]               # Идентификаторы датчиков
    block0_items = file['speed']['block0_items'][:] # Идентификаторы датчиков
    axis1 = file['speed']['axis1'][:]               # Метки времени
    timestamps = pd.to_datetime(axis1)              # Преобразование меток времени в формат datetime
    speed_data = file['speed']['block0_values'][:]  # Данные замеров скорости

pems_bay = pd.DataFrame(speed_data, index=timestamps, columns=axis0)

In [44]:
# Открытие .pkl файла
with open('data/raw_data_without_large_scale_datasets/raw_data/PEMS-BAY/adj_PEMS-BAY.pkl', 'rb') as file:
    data = pickle.load(file, encoding='bytes')
node_ids = [x.decode('utf-8') for x in data[0]]                     # Получаем список id узлов из data[0]
adj_matrix = data[2]                                                # Получаем матрицу смежности из data[2]
adj_df = pd.DataFrame(adj_matrix, index=node_ids, columns=node_ids) # Создание DataFrame с использованием id узлов как индексов и названий колонок
adj_df.columns = [int(i) for i in adj_df.columns]                   # Сопостовление названий

In [45]:
pems_bay.head(1), adj_df.head(1)

(            400001  400017  400030  400040  400045  400052  400057  400059  \
 2017-01-01    71.4    67.8    70.5    67.4    68.8    66.6    66.8    68.0   
 
             400065  400069  ...  409525  409526  409528  409529  413026  \
 2017-01-01    66.8    69.0  ...    68.8    67.9    68.8    68.0    69.2   
 
             413845  413877  413878  414284  414694  
 2017-01-01    68.9    70.4    68.8    71.1    68.0  
 
 [1 rows x 325 columns],
         400001  400017  400030  400040  400045  400052  400057  400059  \
 400001     1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
 
         400065  400069  ...  409525  409526  409528  409529  413026  413845  \
 400001     0.0     0.0  ...     0.0     0.0     0.0     0.0     0.0     0.0   
 
         413877  413878  414284  414694  
 400001     0.0     0.0     0.0     0.0  
 
 [1 rows x 325 columns])

In [145]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import fft

class DFDGCN(nn.Module):
    def __init__(self, num_nodes, seq_len, pre_len, embedding_dim, time_embedding_dim, adj_matrix, adaptive_adj_matrix, logger):
        super(DFDGCN, self).__init__()
        self.num_nodes = num_nodes
        self.seq_len = seq_len
        self.pre_len = pre_len
        self.embedding_dim = embedding_dim
        self.time_embedding_dim = time_embedding_dim
        self.logger = logger
        
        # Identity Embedding
        self.identity_embedding = nn.Embedding(num_nodes, embedding_dim)
        
        # Time Embedding
        self.time_embedding_week = nn.Embedding(7, time_embedding_dim)  # Day of week
        self.time_embedding_hour = nn.Embedding(24, time_embedding_dim)  # Hour of day
        
        # 1D Convolution for feature fusion
        self.conv1d = nn.Conv1d(embedding_dim + time_embedding_dim * 2 + 1, 30, kernel_size=1)
        
        # Learnable parameters for adjacency matrix
        self.W_adj = nn.Parameter(torch.randn(30, 30))
        
        # Static graphs
        self.P = nn.Parameter(adj_matrix, requires_grad=False)  # Predefined graph
        self.A_adt = nn.Parameter(adaptive_adj_matrix, requires_grad=False)  # Adaptive graph
        
        # Graph Convolution Layers
        self.graph_conv = nn.ModuleList([
            nn.Conv2d(1, 1, kernel_size=(1, 3)) for _ in range(3)  # One for each graph type
        ])
        
        # Temporal processing (Causal Convolution)
        self.temporal_conv = nn.Conv1d(seq_len, pre_len, kernel_size=3, padding=1)
        
        # Fully connected layer for final prediction
        self.fc = nn.Linear(30, pre_len)
    
    def forward(self, X, time_features):
        batch_size, seq_len, num_nodes = X.shape
        log_tensor_shape(self.logger, X, "Input X")
        
        # Step 1: Fourier Transform
        F_t = torch.fft.rfft(X, dim=1)  # [batch, seq_len, num_nodes]
        F_t = torch.abs(F_t)  # [batch, seq_len, num_nodes]
        log_tensor_shape(self.logger, F_t, "Fourier Transform F_t")
        
        # Step 2: Identity Embedding
        E_t = self.identity_embedding(torch.arange(num_nodes).to(X.device))  # [num_nodes, embedding_dim]
        log_tensor_shape(self.logger, E_t, "torch.arange(num_nodes) E_t")
        E_t = E_t.unsqueeze(0).unsqueeze(0).expand(batch_size, seq_len, -1, -1)  # [batch, seq_len, num_nodes, embedding_dim]
        log_tensor_shape(self.logger, E_t, "Identity Embedding E_t")
        
        # Step 3: Time Embedding
        day_of_week = time_features[:, :, 0]  # [batch, seq_len]
        hour_of_day = time_features[:, :, 1]  # [batch, seq_len]
        T_week = self.time_embedding_week(day_of_week)  # [batch, seq_len, time_embedding_dim]
        T_hour = self.time_embedding_hour(hour_of_day)  # [batch, seq_len, time_embedding_dim]
        log_tensor_shape(self.logger, T_week, "Embedding T_week")
        log_tensor_shape(self.logger, T_hour, "Embedding T_hour")

        # Масштабируем временные эмбеддинги на все узлы (нужен размер [batch, seq_len, num_nodes, time_embedding_dim])
        T_week = T_week.unsqueeze(-2).expand(-1, -1, self.num_nodes, -1)  # [batch, seq_len, num_nodes, time_embedding_dim]
        T_hour = T_hour.unsqueeze(-2).expand(-1, -1, self.num_nodes, -1)  # [batch, seq_len, num_nodes, time_embedding_dim]
        log_tensor_shape(self.logger, T_week, "Embedding expand T_week")
        log_tensor_shape(self.logger, T_hour, "Embedding expand T_hour")

        T_t = torch.cat([T_week, T_hour], dim=-1)  # [batch, seq_len, num_nodes, time_embedding_dim * 2]
        log_tensor_shape(self.logger, T_t, "Time Embedding T_t")
        
        # Step 4: Concatenate all features
        F_t_expanded = F_t.unsqueeze(-1)  # [batch, seq_len, num_nodes, 1]
        log_tensor_shape(self.logger, F_t_expanded, "Unsqueeze F_t_expanded")

        DE_t = torch.cat([F_t_expanded, E_t, T_t], dim=-1)  # [batch, seq_len, num_nodes, 1 + embedding_dim + time_embedding_dim * 2]
        log_tensor_shape(self.logger, DE_t, "Concatenated DE_t")
        
        # Step 5: 1D Convolution for feature fusion
        DE_t = DE_t.permute(0, 3, 1, 2)  # [batch, channels, seq_len, num_nodes]
        log_tensor_shape(self.logger, DE_t, "Permute DE_t")

        # Преобразуем DE_t в 3D тензор для conv1d, объединяя seq_len и num_nodes
        DE_t = DE_t.view(batch_size, 35, -1)  # [batch, channels, seq_len * num_nodes]
        log_tensor_shape(self.logger, DE_t, "View DE_t")

        DE_t = self.conv1d(DE_t)  # [batch, 30, seq_len * num_nodes]
        log_tensor_shape(self.logger, DE_t, "1D Convolution DE_t")
        
        # Шаг 6: Генерация динамической матрицы смежности
        DE_t = DE_t.view(batch_size, 30, seq_len, num_nodes)  # [batch, 30, seq_len, num_nodes]
        log_tensor_shape(self.logger, DE_t, "View DE_t")

        # Уплощение для матричных операций
        DE_t_flat = DE_t.permute(0, 2, 3, 1).reshape(batch_size * seq_len, num_nodes, -1)  # [batch * seq_len, num_nodes, 30]

        # Генерация динамической матрицы A_t_D
        A_t_D = torch.matmul(DE_t_flat, self.W_adj)  # [batch * seq_len, num_nodes, 30]
        A_t_D = torch.matmul(A_t_D, DE_t_flat.transpose(-1, -2))  # [batch * seq_len, num_nodes, num_nodes]

        # Возвращаем A_t_D в нужный формат
        A_t_D = A_t_D.view(batch_size, seq_len, num_nodes, num_nodes)  # [batch, seq_len, num_nodes, num_nodes]
        A_t_D = F.relu(A_t_D)
        A_t_D = F.softmax(A_t_D, dim=-1)
        log_tensor_shape(self.logger, A_t_D, "Dynamic Adjacency Matrix A_t_D")
        
        # Step 7: Combine Graphs
        P = self.P.unsqueeze(0).unsqueeze(0).expand(batch_size, seq_len, -1, -1)  # [batch, seq_len, num_nodes, num_nodes]
        A_adt = self.A_adt.unsqueeze(0).unsqueeze(0).expand(batch_size, seq_len, -1, -1)  # [batch, seq_len, num_nodes, num_nodes]
        log_tensor_shape(self.logger, P, "Predefined Graph P")
        log_tensor_shape(self.logger, A_adt, "Adaptive Graph A_adt")
        
        # Объединение графов
        Z_t = 0
        for k in range(3):
            if k == 0:
                graph = P
            elif k == 1:
                graph = A_adt
            else:
                graph = A_t_D  # Теперь размерности согласованы
            
            DE_t_transposed = DE_t.permute(0, 2, 3, 1)  # [batch, seq_len, num_nodes, 30]
            Z_t += torch.matmul(graph, DE_t_transposed)  # [batch, seq_len, num_nodes, 30]
        log_tensor_shape(self.logger, Z_t, "Graph Convolution Z_t")
        
        # Step 8: Temporal Processing (Causal Convolution)
        # Перестановка осей и преобразование для Conv1d
        Z_t = Z_t.permute(0, 2, 1, 3)  # [batch, channels, seq_len, num_nodes]
        Z_t = Z_t.reshape(batch_size * num_nodes, seq_len, 30)  # [batch * num_nodes, channels, seq_len]
        log_tensor_shape(self.logger, Z_t, "Reshaped for Temporal Conv Z_t")

        # Применение каузальной свертки
        Z_t = self.temporal_conv(Z_t)  # [batch * num_nodes, channels, pre_len]
        log_tensor_shape(self.logger, Z_t, "Temporal Convolution Z_t")

        # Восстановление исходной размерности
        # Z_t = Z_t.view(batch_size, num_nodes, -1, self.pre_len).permute(0, 3, 1, 2)  # [batch, pre_len, num_nodes, 30]
        Z_t = Z_t.view(batch_size, num_nodes, self.pre_len, -1).permute(0, 2, 1, 3)  # [batch, pre_len, num_nodes, 30]
        log_tensor_shape(self.logger, Z_t, "Restored Z_t")

        # Step 9: Final Prediction
        Y_t = self.fc(Z_t)  # Усредняем последние скрытые каналы: [batch, pre_len, num_nodes]
        Y_t = Y_t.mean(dim=-1)
        log_tensor_shape(self.logger, Y_t, "Final Prediction Y_t")
        
        return Y_t

# Example usage
num_nodes = 325
seq_len = 12
pre_len = 3
batch_size = 64
embedding_dim = 10
time_embedding_dim = 12
adj_matrix = torch.randn(num_nodes, num_nodes)
adaptive_adj_matrix = torch.randn(num_nodes, num_nodes)

# Настройка логгера
logger = setup_logger("dfdgnn_log.txt", terminal=False)

model = DFDGCN(num_nodes, seq_len, pre_len, embedding_dim, time_embedding_dim, adj_matrix, adaptive_adj_matrix, logger)

# Input data
X = torch.randn(batch_size, seq_len, num_nodes)
time_features = torch.randint(0, 7, (batch_size, seq_len, 2))

# Forward pass
output = model(X, time_features)
print(output.shape)  # [64, 12, 325]

RuntimeError: Sizes of tensors must match except in dimension 3. Expected size 7 but got size 12 for tensor number 1 in the list.

### Методология

#### 1. **Преобразование Фурье**
   - **Проблема Time-Shift**: В статье утверждается, что проблема Time-Shift (сдвига во времени) затрудняет моделирование пространственной зависимости в данных о трафике. Для решения этой проблемы предлагается использовать преобразование Фурье, которое переводит данные о трафике в частотную область.
   - **Математическое обоснование**: 
     - Пусть $ f(t) $ — данные о трафике, захваченные сенсорами на определенном перекрестке. Если трафик задерживается на время $ t_0 $, то данные $ f(t-t_0) $ будут захвачены на следующем перекрестке.
     - Согласно определению преобразования Фурье:
       $$
       F(\omega) = \int_{-\infty}^{\infty} f(t) e^{-j\omega t} dt
       $$
       $$
       F_{t_0}(\omega) = \int_{-\infty}^{\infty} f(t-t_0) e^{-j\omega t} dt
       $$
     - После преобразований доказывается, что $ F_{t_0}(\omega) = F(\omega) \cdot e^{-j\omega t_0} $, что означает, что данные в частотной области представлены в одной фазовой размерности, что упрощает изучение пространственных зависимостей между сенсорами.

#### 2. **DFDGCN (Dynamic Frequency Domain Graph Convolutional Network)**
   - **Модуль частотной области**: Основная идея DFDGCN заключается в обновлении динамической матрицы смежности $ A_D $ на основе данных о трафике, наблюдаемых в текущем окне наблюдения.
   - **Шаги модуля**:
     1. **Преобразование Фурье**: Данные о трафике $ X_t $ в каждом окне наблюдения переводятся в частотную область с помощью быстрого преобразования Фурье (FFT):
        $$
        F_t = FFT(X_t)
        $$
     2. **Встраивание идентичности и времени**: Для уменьшения влияния шума в данных о трафике вводятся встраивания идентичности сенсоров $ E_t $ и временные встраивания $ T^{W}_t $ (день недели) и $ T^{D}_t $ (час дня):
        $$
        DE_t = W_{F,t} \cdot F_t || E_t || W_{T,t} \cdot (T^{W}_t || T^{D}_t)
        $$
     3. **Одномерная свертка**: Применяется одномерная сверточная слой с ядром $ 1 \times 1 $ для дополнительного встраивания, чтобы изучить связи между измерениями $ DE_t $.
     4. **Полносвязный слой и матричное умножение**: Применяется полносвязный слой для изучения $ DE_t $, затем результат умножается на транспонированную матрицу $ DE_t $. После активационной функции и $ Softmax $ получается финальная матрица смежности:
        $$
        A^{t}_D = Softmax(ReLU(DE_t W_{adj} DE^{T}_t))
        $$
     5. **Граф сверточный слой**: Динамический частотный граф комбинируется с предопределенными графами $ P $ (из DCRNN) и самоадаптивными графами $ A_{adt} $ (из GWNet) для создания графового сверточного слоя:
        $$
        Z_t = \sum_{k=0}^{K} (P^{k} X_t W_{k,1} + A^{k}_{adt} X_t W_{k,2} + A^{t}_D X_t W_{k,3})
        $$
     6. **Обработка временной информации**: Для обработки временной информации используется классическая причинная свертка с остаточной сетью, как в GWNet.

#### 3. **Эксперименты**
   - **Датасеты**: Эксперименты проводятся на четырех реальных датасетах с десятками тысяч временных шагов и сотнями сенсоров. Статистика датасетов представлена в таблице 1.
   - **Базовые линии и метрики**: В качестве базовых линий выбраны классические методы, такие как HI, GWNet, DCRNN, AGCRN, STGCN, MTGNN, DGCRN. Метрики оценки включают среднюю абсолютную ошибку (MAE), среднеквадратичную ошибку (RMSE) и среднюю абсолютную процентную ошибку (MAPE).
   - **Настройки экспериментов**: Датасеты делятся на обучающую, валидационную и тестовую выборки в соотношении 7:1:2. Предсказываются данные о трафике на 12 временных шагов с использованием исторических данных длиной 12. Размеры встраиваний после преобразования Фурье и идентичности составляют 10, а временные встраивания $ T^{W}_t $ и $ T^{D}_t $ — 12. Размер встраивания после одномерной свертки — 30.
   - **Результаты экспериментов**: DFDGCN показывает лучшие результаты по сравнению с базовыми линиями на всех датасетах. Анализ аблационных экспериментов подтверждает эффективность частотного графа в моделировании динамической пространственной зависимости.

In [153]:
import torch
import torch.nn.functional as F

# Параметры
batch_size = 32  # Размер батча
seq_len = 12     # Длина последовательности (количество временных шагов)
num_nodes = 2   # Количество узлов (сенсоров)

# Создаем случайные данные в формате [batch, seq_len, num_nodes]
traffic_data = torch.randn(batch_size, seq_len, num_nodes)

# Применяем преобразование Фурье
# В PyTorch преобразование Фурье можно выполнить с помощью torch.fft.fft
# Мы применяем его по оси времени (seq_len)
traffic_data_fft = torch.fft.fft(traffic_data, dim=1)

# Выводим результат
print("Исходные данные (временная область):")
print(traffic_data.shape)
print("\nДанные после преобразования Фурье (частотная область):")
print(traffic_data_fft.shape)

Исходные данные (временная область):
torch.Size([32, 12, 2])

Данные после преобразования Фурье (частотная область):
torch.Size([32, 12, 2])


In [189]:
import torch
import torch.nn.functional as F

# Параметры
batch_size = 32  # Размер батча
seq_len = 12     # Длина последовательности (количество временных шагов)
num_nodes = 20   # Количество узлов (сенсоров)
embedding_dim = 10  # Размерность эмбеддингов

# Создаем случайные данные в формате [batch, seq_len, num_nodes]
traffic_data = torch.randn(batch_size, seq_len, num_nodes)

# 1. Преобразование Фурье
# F_t = FFT(X_t)
F_t = torch.fft.fft(traffic_data, dim=1)
F_t = torch.abs(F_t)

# 2. Эмбеддинг идентичности сенсоров
# E_t: [batch, num_nodes, embedding_dim]
idntity_eembedding = torch.randn(batch_size, num_nodes, embedding_dim)

# 3. Временные метки (день недели и час дня)
# T^{W}_t: [batch, seq_len, 1] (меток дня недели)
# T^{D}_t: [batch, seq_len, 1] (меток часа дня)
day_of_week = torch.randint(0, 7, (batch_size, seq_len, 1))  # Метки дня недели
hour_of_day = torch.randint(0, 288, (batch_size, seq_len, 1))  # Метки часа дня

# 4. Линейное преобразование частотных данных
# W_{F,t} \cdot F_t
W_F_t = torch.randn(num_nodes, embedding_dim)  # Обучаемый вес для частотных данных
F_t_transformed = torch.matmul(W_F_t.permute(0, 2, 1), F_t) # [batch, seq_len, embedding_dim]
print(f'{F_t.shape, W_F_t.shape = }')

# 5. Линейное преобразование временных меток
# W_{T,t} \cdot (T^{W}_t || T^{D}_t)
W_T_t = torch.randn(7 + 288, embedding_dim)  # Обучаемый вес для временных меток

# Конкатенация временных меток
temporal_labels = torch.cat([day_of_week, hour_of_day], dim=-1)  # [batch, seq_len, 7 + 288]

# Линейное преобразование временных меток
temporal_embedding = torch.matmul(temporal_labels.float(), W_T_t)  # [batch, seq_len, embedding_dim]

# 6. Конкатенация
# DE_t = W_{F,t} \cdot F_t || E_t || W_{T,t} \cdot (T^{W}_t || T^{D}_t)
# Расширяем размерности для конкатенации
F_t_transformed_expanded = F_t_transformed.unsqueeze(2)  # [batch, seq_len, 1, embedding_dim]
identity_embedding_expanded = identity_embedding.unsqueeze(1).expand(-1, seq_len, -1, -1)  # [batch, seq_len, num_nodes, embedding_dim]
temporal_embedding_expanded = temporal_embedding.unsqueeze(2)  # [batch, seq_len, 1, embedding_dim]

# Конкатенация
DE_t = torch.cat([F_t_transformed_expanded, identity_embedding_expanded, temporal_embedding_expanded], dim=-1)

# Выводим результат
print("Результат конкатенации DE_t:")
print(DE_t.shape)  # Ожидаемый размер: [batch, seq_len, num_nodes, embedding_dim + embedding_dim + embedding_dim]

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3